# <center> Проект: Анализ вакансий из HeadHunter
   

In [107]:
import pandas as pd
from sqlalchemy import create_engine as sql_ce, text as sql_text
from config import *
import requests
from bs4 import BeautifulSoup as BS

In [108]:
# подключение к БД
connection = sql_ce(f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}')

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [1]:
# текст запроса
query_3_1 = f'''
                select 
                    count(*)
                from
                    vacancies
            '''

In [8]:
# результат запроса
# df = pd.read_sql_query(query, connection) # не работает
df = pd.DataFrame(connection.connect().execute(sql_text(query_3_1)))
print('Количество вакансий в базе данных: ', end='')
print(df['count'][0])

Количество вакансий в базе данных: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [10]:
# текст запроса
query_3_2 = f'''
                select
                    count(*)
                from 
                    employers
            '''

In [11]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_3_2)))
print('Количество работодателей в базе данных: ', end='')
print(df['count'][0])

Количество работодателей в базе данных: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [12]:
# текст запроса
query_3_3 = f'''
                select 
                    count(*)
                from 
                    areas
            '''

In [13]:
# результат запроса 
df = pd.DataFrame(connection.connect().execute(sql_text(query_3_3)))
print('Количество регионов в базе данных: ', end='')
print(df['count'][0])

Количество регионов в базе данных: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [16]:
# текст запроса
query_3_4 = f'''
                select 
                    count(*)
                from 
                    industries
            '''

In [17]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_3_4)))
print('Количество сфер деятельности в базе данных: ', end='')
print(df['count'][0])

Количество сфер деятельности в базе данных: 294


***

In [208]:
query_3_5 = f'''
                select 
                    count(*)
                from 
                    industries
            '''
df = pd.DataFrame(connection.connect().execute(sql_text(query_3_5)))
count_industries = df['count'][0]

query_3_6 = f'''
                select 
                    count(*)
                from 
                    employers
            '''
df = pd.DataFrame(connection.connect().execute(sql_text(query_3_6)))
count_employers = df['count'][0]

print('Среднее количество работодателей работающий в одной сфере: ', end='')
print(round(count_employers/count_industries))

Среднее количество работодателей работающий в одной сфере: 80


In [218]:
query_3_7 = f'''
                select 
                    count(*)
                from 
                    vacancies
            '''
df = pd.DataFrame(connection.connect().execute(sql_text(query_3_7)))
count_vacancies = df['count'][0]

query_3_8 = f'''
                select 
                    count(*)
                from 
                    employers
            '''
df = pd.DataFrame(connection.connect().execute(sql_text(query_3_8)))
count_employers = df['count'][0]

print('Среднее количество ваканисий на одного работодателя: ', end='')
print(round(count_vacancies/count_employers))

Среднее количество ваканисий на одного работодателя: 2


### Вывод
    1. В среднем на каждого работодателя приходится по 2 вакансии
    2. В среднем в одной сфере деятельности работает около 80 работодателей

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [88]:
# текст запроса
query_4_1 = f'''
                -- Обобщенное табличное выражение
                -- Получаем таблицу сгрупированную по названию регионов
                
                with x as 
                (
                    select 
                            a.name area,
                            count(v.id) cnt
                    from
                        vacancies v
                        join areas a on a.id = v.area_id
                    group by 
                        area
                    order by 
                        cnt desc
                )
                
                -- Добавляем процент вакансий в регионах от общего количества вакансий
                
                select
                    *,
                    round(cnt * 100 / (select sum(cnt) from x))||'%' percent
                from
                    x
                limit 5
            '''

In [89]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_4_1)))
print('Количество вакансий в зависимости от региона:')
df

Количество вакансий в зависимости от региона:


,area,cnt,percent
0,Москва,5333,11%
1,Санкт-Петербург,2851,6%
2,Минск,2112,4%
3,Новосибирск,2006,4%
4,Алматы,1892,4%


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [90]:
# текст запроса
query_4_2 = f'''
                
                select 
                    count(*) cnt,
                    -- Вычисляем процент от общего числа вакансий
                    round(count(*) * 100 / (select count(*) from vacancies))||' %' percent
                from
                    vacancies v
                where
                    -- Фильтруем вакансии по заработной плате
                    v.salary_from is not null or v.salary_to is not null
                
            '''

In [91]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_4_2)))
print('Количество вакансий в которых указана заработная плата: ', end='')
print(df['cnt'][0])
df

Количество вакансий в которых указана заработная плата: 24073


,cnt,percent
0,24073,48 %


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [25]:
# текст запроса
query_4_3 = f'''
                select
                    round(avg(salary_from)) avg_salary_from,
                    round(avg(salary_to)) avg_salary_to
                from
                    vacancies
            '''

In [26]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_4_3)))
df
print('Среднее значение нижней границы заработной платы: ', end='')
print(df['avg_salary_from'][0])
print('Среднее значение верхней границы заработной платы: ', end='')
print(df['avg_salary_to'][0])

Среднее значение нижней границы заработной платы: 71065
Среднее значение верхней границы заработной платы: 110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [27]:
# текст запроса
query_4_4 = f'''
                
                with x as
                (
                    select
                        schedule,
                        employment,
                        count(id) cnt
                    from
                        vacancies
                    group by 
                        schedule, 
                        employment
                    order by 
                        cnt desc
                )
                
                select
                    *,
                    round(cnt * 100 / (select sum(cnt) from x)) || ' %' percent
                from
                    x
            '''

In [28]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_4_4)))
print('Количество вакансий по типу трудоустройства и рабочему графику:')
df

Количество вакансий по типу трудоустройства и рабочему графику:


,schedule,employment,cnt,percent
0,Полный день,Полная занятость,35367,72%
1,Удаленная работа,Полная занятость,7802,16%
2,Гибкий график,Полная занятость,1593,3%
3,Удаленная работа,Частичная занятость,1312,3%
4,Сменный график,Полная занятость,940,2%
5,Полный день,Стажировка,569,1%
6,Вахтовый метод,Полная занятость,367,1%
7,Полный день,Частичная занятость,347,1%
8,Гибкий график,Частичная занятость,312,1%
9,Полный день,Проектная работа,141,0%


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [92]:
# текст запроса
query_4_5 = f'''
                with x as
                (
                    select
                        experience,
                        count(id) cnt
                    from
                        vacancies
                    group by
                        experience
                    order by
                        cnt
                )
                
                select
                    experience,
                    cnt,
                    round(cnt * 100 / (select sum(cnt) from x)) || ' %' percent
                from
                    x
            '''

In [93]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_4_5)))
print('Количество вакансий от требуемого опыта работы:')
df

Количество вакансий от требуемого опыта работы:


,experience,cnt,percent
0,Более 6 лет,1337,3 %
1,Нет опыта,7197,15 %
2,От 3 до 6 лет,14511,29 %
3,От 1 года до 3 лет,26152,53 %


***

In [94]:
# текст запроса
query_4_6 = f'''
                -- Обобщеное табличное выражение
                -- Находим средне значение зарплаты

                 with x as
                (
                    select
                        *,
                        coalesce(salary_from, 0) + coalesce(salary_to, 0) salary
                    from
                        vacancies
                    where
                        -- Если присутствует либо верняя, либо нижняя граница зарплаты
                        not (salary_from is not null and salary_to is not null) and
                        (salary_from is not null or salary_to is not null)
                    
                    union all
                    
                    select
                        *,
                        (coalesce(salary_from, 0) + coalesce(salary_to, 0))/2
                    from
                        vacancies
                    where
                        -- Когда присутствуют обе границы зарплаты
                        (salary_from is not null and salary_to is not null)
                )
                ------------
                
                select
                    schedule,
                    employment,
                    count(id) cnt,
                    round(avg(salary)) avg_salary
                from
                    x
                group by
                    schedule,
                    employment
                order by
                    cnt desc,
                    schedule,
                    employment
                
            '''
df = pd.DataFrame(connection.connect().execute(sql_text(query_4_6)))
print('Количество вакансий от требуемого опыта работы с указанием средней заработной платы:')
df

Количество вакансий от требуемого опыта работы с указанием средней заработной платы:


,schedule,employment,cnt,avg_salary
0,Полный день,Полная занятость,16848,81119
1,Удаленная работа,Полная занятость,3713,120753
2,Удаленная работа,Частичная занятость,923,26171
3,Сменный график,Полная занятость,715,45911
4,Гибкий график,Полная занятость,650,118982
5,Вахтовый метод,Полная занятость,269,123693
6,Полный день,Стажировка,264,36192
7,Полный день,Частичная занятость,192,50880
8,Гибкий график,Частичная занятость,148,39832
9,Сменный график,Частичная занятость,97,32061


In [33]:
# текст запроса
query_4_7 = f'''
                with x as
                (
                    select
                        *,
                        coalesce(salary_from, 0) + coalesce(salary_to, 0) salary
                    from
                        vacancies
                    where 
                        not (salary_from is not null and salary_to is not null) and
                        (salary_from is not null or salary_to is not null)
                    
                    union all
                    
                    select
                        *,
                        (coalesce(salary_from, 0) + coalesce(salary_to, 0))/2
                    from
                        vacancies
                    where
                        (salary_from is not null and salary_to is not null)
                )
                ------------
                
                select
                    ar.name,
                    count(x.id) cnt,
                    round(avg(x.salary)) avg_salary
                from
                    x x
                    join areas ar on ar.id = x.area_id
                group by
                    ar.name
                having
                    count(x.id) >= 10
                order by
                    avg_salary desc
                limit 20
            '''
df = pd.DataFrame(connection.connect().execute(sql_text(query_4_7)))
print('Количество вакансий с указанием средней заработной платы по регионам (более 10 вакансий):')
df

Количество вакансий с указанием средней заработной платы по регионам (более 10 вакансий):


,name,cnt,avg_salary
0,Турция,19,235435
1,Кипр,39,223736
2,Армения,106,219570
3,Сербия,12,208423
4,Тбилиси,80,207156
5,Батуми,18,199834
6,Иннополис,17,175882
7,Баку,51,155747
8,Москва,1592,134140
9,Лобня,11,121136


### Вывод:
    1. Большинство вакансий размещено в городах Москва и Санкт-Петербург, 11% и 6% соответсвенно.
    2. Около половины вакансий размещено без указания заработной платы
    3. 72% всех ваканчий приходятся на полный день и полную занятость. Только 16% относятся к удаленной работе.
    4. Половина всех вакансий требуют опыт работы от 1 года до 3 лет, треть - от 3 до 6 лет.
    5. Удаленная работа при полной занятости оплачивается выше чем полный день полной занятости.
    7. Москва находится на 9 месте среди всех регионов опубликованных вакансий. Регионы где средняя заработная плата выше чем в Москве находятся за границей.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [34]:
# текст запроса
query_5_1 = f'''
                select
                    e.name "Работодатели",
                    count(v.id) cnt,
                    round((count(*) * 10000 / (select count(*) from vacancies)))/100||'%' percent
                from
                    vacancies v
                    join employers e on e.id = v.employer_id
                group by 
                    "Работодатели"
                order by 
                    cnt desc
                limit 10
            '''

In [35]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_5_1)))
print('Количество вакансий на работодателя:')
df

Количество вакансий на работодателя:


,Работодатели,cnt,percent
0,Яндекс,1933,3.92%
1,Ростелеком,491,0.99%
2,Тинькофф,444,0.9%
3,СБЕР,428,0.86%
4,Газпром нефть,331,0.67%
5,ИК СИБИНТЕК,327,0.66%
6,МТС,292,0.59%
7,DataArt,247,0.5%
8,Совкомбанк Технологии,204,0.41%
9,Первый Бит,176,0.35%


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [94]:
# текст запроса
query_5_2 = f'''
                -- Обобщеное табличное выражение
                -- Получаем сгруппированную таблицу по регионам от количества работодателей.
                
                with ar as
                (
                    select
                        a.name name_area,
                        count(e.id) count_employer
                    from
                        employers e
                        join areas a on a.id = e.area
                    group by 
                        a.name
                ),
                
                -- Получаем сгруппированную таблицу по регионам от количества вакансий.
                
                em as
                (
                    select
                        a.name name_area,
                        count(v.id) count_vacancies
                    from
                        vacancies v
                        left join areas a on a.id = v.area_id
                    group by 
                        a.name
                )
                
                -- Объединяем две таблицы поо названию региона
                
                select
                    ar.name_area,
                    ar.count_employer,
                    em.count_vacancies
                from 
                    ar ar
                    left join em em on em.name_area = ar.name_area
                where 
                    em.count_vacancies is null
                order by 
                    ar.count_employer desc
                limit 5
            '''

In [95]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_5_2)))
df

,name_area,count_employer,count_vacancies
0,Россия,410,None
1,Казахстан,207,None
2,Московская область,75,None
3,Краснодарский край,19,None
4,Беларусь,18,None


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [36]:
# текст запроса
query_5_3 = f'''
                
                select
                    e.name "Работодатель",
                    count(distinct v.area_id) cnt
                from
                    vacancies v
                    left join employers e on e.id = v.employer_id
                group by 
                    e.name
                order by 
                    cnt desc
                limit 3
                
            '''

In [37]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_5_3)))
print('Количество регионов в которых публикует вакансии работодатель:')
df

Количество регионов в которых публикует вакансии работодатель:


,Работодатель,cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [95]:
# текст запроса
query_5_4 = f'''
                -- Количество вакансий без сферы деятельности
                with x as
                (
                    select
                        count(*) cnt
                    from
                        employers em
                        left join employers_industries ei on ei.employer_id = em.id
                    where 
                        ei.employer_id is null
                )
                
                -- Процент эттих вакансий от общего числа
                
                select
                    *,
                    cnt * 100 / (select count(*) from employers)||' %' percent
                from
                    x x
                
            '''

In [96]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_5_4)))
print('Количество работодателей не указавших сферу деятельности:')
df

Количество работодателей не указавших сферу деятельности:


,cnt,percent
0,8419,35 %


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [32]:
# текст запроса
query_5_5 = f'''
                
                select
                    em.name "Название компании",
                    count(em.id) "кол-во сфер деятельности"
                from
                    employers_industries ei
                    left join employers em on ei.employer_id = em.id
                group by 
                    em.name
                having 
                    count(em.id) = 4
                order by 
                    em.name
                limit 5
            '''

In [33]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_5_5)))
df

,Название компании,кол-во сфер деятельности
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4


In [41]:
# текст запроса
query_5_5_1 = f'''
                
                select
                    em.name "Название компании",
                    count(em.id) "кол-во сфер деятельности"
                from
                    employers_industries ei
                    left join employers em on ei.employer_id = em.id
                group by 
                    em.name
                order by 
                    "кол-во сфер деятельности" desc
                limit 10
            '''
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_5_5_1)))
print('Количество сфер деятельности у компании:')
df

Количество сфер деятельности у компании:


,Название компании,кол-во сфер деятельности
0,Модуль,16
1,Простые решения,11
2,Энергия,10
3,Прайд,10
4,СБК,9
5,Альфа,9
6,Эксперт,9
7,Бизнес-Ассистанс,8
8,Уфанет,8
9,Автомастер,8


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [42]:
# текст запроса
query_5_6 = f'''
                -- Группировка по сферам деяттельности от количества вакансий
                
                select
                    i.name "Сфера деятельности",
                    count(em.id) "Количество компаний"
                from
                    employers_industries ei
                    left join employers em on ei.employer_id = em.id
                    left join industries i on i.id = ei.industry_id
                group by 
                    i.name
                having 
                    i.name like 'Разработка программного обеспечения'
            '''

In [43]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_5_6)))
df

,Сфера деятельности,Количество компаний
0,Разработка программного обеспечения,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [44]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8#%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D0%BE%D0%B2'
req = requests.get(url=url)
page = BS(req.text, 'html.parser')

# Находим таблицу с городами миллиониками на сайте
citys = page.find('table',  class_='standard sortable').find('tbody')
citys = citys.find_all('tr')

# Составим список из имен городов миллиоников
citys_list = []
city: BS
for city in citys[1:]:
    citys_list.append(city.find('td').find_next_sibling('td').find('a').get('title', 'no title'))
#  Преобразуем список в строку, которую можно использовать в SQL запросе
citys_list = str(tuple(citys_list))


In [46]:
# текст запроса
city = 'Яндекс'
query_5_7 = f'''
                -- Получаем сгруппиррованную таблицу по городам и компаниям от количества вакансий
                -- и выбираем нужную компанию

                with citys as
                (
                    select
                        em.name "Компания",
                        ar.name "Город",
                        count(v.id) cnt
                    from
                        vacancies v
                        join employers em on em.id = v.employer_id
                        join areas ar on ar.id = v.area_id
                    group by 
                        em.name, 
                        ar.name
                    having 
                        em.name = '{city}'
                )
                
                -- Согласно списку городов миллионников выбираем только те города, которые есть в списке
                
                (select 
                    "Город",
                    cnt
                from citys
                where 
                    "Город" in {citys_list}
                order by 
                    cnt desc)
                
                union all
                
                -- И добавляем строку 'Total' с суммой вакансий 
                
                select
                    'Total' "Город",
                    sum(cnt) cnt
                from
                    citys
                where 
                    "Город" in {citys_list}
                
            '''

In [48]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_5_7)))
print('Количество вакансий компании Яндекс в городах милионниках:')
df

Количество вакансий компании Яндекс в городах милионниках:


,Город,cnt
0,Москва,54
1,Санкт-Петербург,42
2,Екатеринбург,39
3,Нижний Новгород,36
4,Новосибирск,35
5,Воронеж,32
6,Краснодар,30
7,Самара,26
8,Уфа,26
9,Казань,25


***

In [97]:
query_5_8 = f'''
                -- Вакансии с указанием заработной платы
                
                with x as
                (
                    select
                        *,
                        coalesce(salary_from, 0) + coalesce(salary_to, 0) salary
                    from
                        vacancies
                    where 
                        not (salary_from is not null and salary_to is not null) and
                        (salary_from is not null or salary_to is not null)
                    
                    union all
                    
                    select
                        *,
                        (coalesce(salary_from, 0) + coalesce(salary_to, 0))/2
                    from
                        vacancies
                    where 
                        (salary_from is not null and salary_to is not null)
                ),
                -------------------
                
                -- Таблица сгруппированная по работодателям от количества
                
                t as
                (
                    select
                        e.name "Работодатели",
                        count(v.id) cnt
                    from
                        vacancies v
                        join employers e on e.id = v.employer_id
                    group by 
                        e.name
                ),
                -------------------
                
                -- Таблица сгруппированная по работодателям от количества с указанием заработной платы
                
                p as
                (              
                select
                    e.name "Работодатели",
                    count(v.id) cnt,
                    round(avg(v.salary)) avg_salary
                from
                    x v
                    join employers e on e.id = v.employer_id
                group by 
                    e.name
                )
                -------------------
                
                -- Объединение двух таблиц, с указанием заработной платы и общей
                
                select
                    p."Работодатели",
                    p.cnt,
                    t.cnt "из",
                    (p.cnt * 100 / t.cnt)||' %' "%",
                    p.avg_salary
                from
                    p p
                    join t t on t."Работодатели" = p."Работодатели"
                order by 
                    p.cnt desc
                limit 20
                
                
            '''
df = pd.DataFrame(connection.connect().execute(sql_text(query_5_8)))
print('Общее количество вакансий, количество вакансий с указанием заработной платы')
print('Средняя заработная плата.')
df

Общее количество вакансий, количество вакансий с указанием заработной платы
Средняя заработная плата.


,Работодатели,cnt,из,%,avg_salary
0,Яндекс,1495,1933,77 %,25884
1,Ростелеком,392,491,79 %,46980
2,Тензор,155,155,100 %,171149
3,Спецремонт,151,151,100 %,70417
4,Первый Бит,132,176,75 %,92324
5,Поляков Денис Иванович,127,127,100 %,52638
6,МТС,92,292,31 %,64521
7,Andersen,86,155,55 %,269475
8,1C-Рарус,77,80,96 %,81548
9,ООО ЕФИН,71,71,100 %,150000


In [50]:
query_5_9 = f'''
                with x as
                (
                    select
                        *,
                        coalesce(salary_from, 0) + coalesce(salary_to, 0) salary
                    from
                        vacancies
                    where 
                        not (salary_from is not null and salary_to is not null) and
                        (salary_from is not null or salary_to is not null)
                    
                    union all
                    
                    select
                        *,
                        (coalesce(salary_from, 0) + coalesce(salary_to, 0))/2
                    from
                        vacancies
                    where 
                        (salary_from is not null and salary_to is not null)
                )
                ------------
                

                select
                    e.name "Работодатели",
                    count(v.id) cnt,
                    round(avg(v.salary)) avg_salary
                from
                    x v
                    join employers e on e.id = v.employer_id
                group by 
                    "Работодатели"
                order by 
                    avg_salary desc
                limit 10
            '''
df = pd.DataFrame(connection.connect().execute(sql_text(query_5_9)))
print('Максимальная средняя заработная плата:')
df

Максимальная средняя заработная плата:


,Работодатели,cnt,avg_salary
0,Alpha Personnel,1,1000000
1,ЦТП,1,650000
2,RuWork,1,555261
3,Петухова Алия,5,552700
4,Агентство недвижимости ЛЕТО,2,550000
5,Пикабу,1,550000
6,BeRecruited,1,550000
7,ВСП Партнер,1,500000
8,"Август, Фирма",10,500000
9,StaffRecruitment,3,483333


### Вывод:
    1. Треть всех компаний не указала сферу деятельности.
    2. Компания модуль имеет 16 сфер деятельности.
    3. Компания Яндекс 10% всех вакансий в городах миллиониках разместила в г. Москва
    4. Яндекс разместила больше всего вакансий, однако средняя заработная плата меньшая среди остальных работодателей
    5. В четверти вакансий компании Яндекс не указана заработная плата.
    6. Самую высокую среднюю заработную плату предоставляет компания 'Andersen', однако только в половина всех вакансий указана заработная плата.
    7. Самая большая заработная плата у вакансии от компании "Alpha Personnel" и составляет 1 000 000 руб.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [52]:
# текст запроса
query_6_1 = f'''
                with x as
                (
                    select
                        *
                    from
                        vacancies v
                    where 
                        lower(v.name) like '%data%' or lower(v.name) like '%данн%'
                )
                
                select 
                    count(*)
                from 
                    x
            '''

In [56]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_6_1)))
print('Количество вакансий содержащие слова "data" и "данн": ', end='')
print(df['count'][0])

Количество вакансий содержащие слова "data" и "данн": 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [98]:
# текст запроса
query_6_2 = f'''
                -- Таблица с вакансиями для DS

                with x as
                (
                    select
                        *
                    from
                        vacancies v
                    where 
                        lower(v.name) like '%data scientist%' or
                        lower(v.name) like '%data science%' or
                        lower(v.name) like '%исследователь данных%' or
                        lower(v.name) like '%machine learning%' or
                        lower(v.name) like '%машинн%обучен%' or
                        (v.name like '%ML%' and v.name not like '%HTML%')
                ), 
                
                -- Фильтруем вакансии для начинающих DS
                
                t as
                (
                select 
                    *
                from
                    x 
                where
                    lower(name) like '%junior%' or
                    experience like 'Нет опыта' or
                    employment like 'Стажировка'
                )
                
                -- Добавляем процент от всех вакансий для DS
                
                select
                    count(*),
                    (count(*) * 100 / (select count(*) from x)) || ' %' percent
                from
                    t
            '''

In [99]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_6_2)))
print('Количество вакансий для начинающего DS: ', end='')
print(df['count'][0])
df

Количество вакансий для начинающего DS: 51


,count,percent
0,51,10 %


In [61]:
query_6_2_1 = f'''
                with x as
                (
                    select
                        *
                    from
                        vacancies v
                    where 
                        lower(v.name) like '%data scientist%' or
                        lower(v.name) like '%data science%' or
                        lower(v.name) like '%исследователь данных%' or
                        lower(v.name) like '%machine learning%' or
                        lower(v.name) like '%машинн%обучен%' or
                        (v.name like '%ML%' and v.name not like '%HTML%')
                ) 
                
                select
                    round((count(*) * 10000/ (select count(*) from vacancies)))/100 || ' %' "%"
                from
                    x
            '''
df = pd.DataFrame(connection.connect().execute(sql_text(query_6_2_1)))
print('Процент вакансий DS: ', end='')
print(df['%'][0])

Процент вакансий DS: 0.97 %


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [63]:
# текст запроса
query_6_3 = f'''
                with x as
                (
                    select
                        *
                    from
                        vacancies v
                    where 
                        lower(v.name) like '%data scientist%' or
                        lower(v.name) like '%data science%' or
                        lower(v.name) like '%исследователь данных%' or
                        lower(v.name) like '%machine learning%' or
                        lower(v.name) like '%машинн%обучен%' or
                        (v.name like '%ML%' and v.name not like '%HTML%')
                ),
                
                -- Фильтруем по ключевым навыкам 'sql' и 'postgres'
                
                t as
                (
                    select
                        *
                    from
                        x
                    where
                        lower(key_skills) like '%sql%' or
                        lower(key_skills) like '%postgres%'
                )
                
                -- Добавляем процент от всех вакансий для DS
                
                select
                    count(*),
                    round(count(*) * 10000 /(select count(*) from x))/100 || ' %' "%"
                from
                    t
            '''

In [64]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_6_3)))
print('Количество вакансий DS со знание SQL: ', end='')
print(df['count'][0])
df

Количество вакансий DS со знание SQL: 201


,count,%
0,201,41.87 %


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [100]:
# текст запроса
query_6_4 = f'''
                with x as
                (
                    select
                        *
                    from
                        vacancies v
                    where 
                        lower(v.name) like '%data scientist%' or
                        lower(v.name) like '%data science%' or
                        lower(v.name) like '%исследователь данных%' or
                        lower(v.name) like '%machine learning%' or
                        lower(v.name) like '%машинн%обучен%' or
                        (v.name like '%ML%' and v.name not like '%HTML%')
                ),
                
                -- Фильтруем по ключевому навыку 'python'
                
                t as
                (
                    select
                        *
                    from
                        x
                    where
                        lower(key_skills) like '%python%'
                )
                
                -- Добавляем процент от всех вакансий для DS
                
                select
                    count(*),
                    round(count(*) * 10000 /(select count(*) from x))/100 || ' %' "%"
                from
                    t
            '''

In [101]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_6_4)))
print('Количество вакансий DS со знание Python: ', end='')
print(df['count'][0])
df

Количество вакансий DS со знание Python: 351


,count,%
0,351,73.12 %


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [102]:
# текст запроса
query_6_5 = f'''
                with x as
                (
                    select
                        *
                    from
                        vacancies v
                    where 
                        lower(v.name) like '%data scientist%' or
                        lower(v.name) like '%data science%' or
                        lower(v.name) like '%исследователь данных%' or
                        lower(v.name) like '%machine learning%' or
                        lower(v.name) like '%машинн%обучен%' or
                        (v.name like '%ML%' and v.name not like '%HTML%')
                ),
                
                -- Находим количество ключевых навыков для вакансий DS
                
                t as
                (
                    select
                        length(key_skills) - length(replace(key_skills, chr(9), '')) + 1 cnt
                    from
                        x
                    where
                        key_skills is not null
                )

                select
                    round(avg(cnt),2) cnt
                from
                    t
            '''

In [103]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_6_5)))
print('Среднее количество ключевых навыков для DS: ', end='')
print(df['cnt'][0])

Среднее количество ключевых навыков для DS: 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [76]:
# текст запроса
query_6_6 = f'''
                -- Ттаблица с вакансиями для DS
                
                 with x as
                (
                    select
                        *
                    from
                        vacancies v
                    where 
                        lower(v.name) like '%data scientist%' or
                        lower(v.name) like '%data science%' or
                        lower(v.name) like '%исследователь данных%' or
                        lower(v.name) like '%machine learning%' or
                        lower(v.name) like '%машинн%обучен%' or
                        (v.name like '%ML%' and v.name not like '%HTML%')
                ),
                ------------
                
                -- Оставляем только те, у котторых указана зарабатная плата
                
                k as
                (
                    select
                        *,
                        coalesce(salary_from, 0) + coalesce(salary_to, 0) salary
                    from
                        x
                    where 
                        not (salary_from is not null and salary_to is not null) and
                        (salary_from is not null or salary_to is not null)
                    
                    union all
                    
                    select
                        *,
                        (coalesce(salary_from, 0) + coalesce(salary_to, 0))/2
                    from
                        x
                    where 
                        (salary_from is not null and salary_to is not null)
                )
                ------------
                
                -- Группируем по опыту работы от средней заработной платы
                
                select
                    experience,
                    round(avg(salary), -3)
                from
                    k
                group by 
                    experience
            '''

In [77]:
# результат запроса
df = pd.DataFrame(connection.connect().execute(sql_text(query_6_6)))
print('Среднее заработная плата DS в зависимости от опыта работа: ')
df

Среднее заработная плата DS в зависимости от опыта работа: 


,experience,round
0,От 3 до 6 лет,243000
1,От 1 года до 3 лет,140000
2,Нет опыта,75000


***

### Вывод:
    1. Среди всех вакансий только 51 вакансия подходит для начинающего DS, что примерно 1%.
    2. Это так же составляет 10% от всех заявок для DS.
    3. Среди всех заявок знание SQL требуется в  42 % случаях, а зание Python - в 73 %.
    4. Среднее количество навыков для DS больше 6.
    5. С увиличением количества опыта возрастает и заработная плата. от 75 000 р для новичка и до 245 000 для опытного специалиста.

# Общий вывод по проекту

In [104]:
# текст запроса
query_7_1 = f'''
                 with x as
                (
                    select
                        *
                    from
                        vacancies v
                    where 
                        lower(v.name) like '%data scientist%' or
                        lower(v.name) like '%data science%' or
                        lower(v.name) like '%исследователь данных%' or
                        lower(v.name) like '%machine learning%' or
                        lower(v.name) like '%машинн%обучен%' or
                        (v.name like '%ML%' and v.name not like '%HTML%')
                )
                
                select
                    e.name "Работодатель",
                    count(k.id) cnt
                from
                    x k
                    join employers e on e.id = k.employer_id
                group by 
                    e.name
                order by
                    cnt desc
                limit 20
            '''

df = pd.DataFrame(connection.connect().execute(sql_text(query_7_1)))
print('Количество вакансий DS по компаниям:')
df

Количество вакансий DS по компаниям:


,Работодатель,cnt
0,СБЕР,37
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11
5,Яндекс,9
6,EvenBet Gaming,9
7,МегаФон,8
8,Andersen,7
9,МТС,6


In [80]:
# текст запроса
query_7_2 = f'''
                 with x as
                (
                    select
                        *
                    from
                        vacancies v
                    where 
                        lower(v.name) like '%data scientist%' or
                        lower(v.name) like '%data science%' or
                        lower(v.name) like '%исследователь данных%' or
                        lower(v.name) like '%machine learning%' or
                        lower(v.name) like '%машинн%обучен%' or
                        (v.name like '%ML%' and v.name not like '%HTML%')
                ),
                ------------
                
                k as
                (
                    select
                        *,
                        coalesce(salary_from, 0) + coalesce(salary_to, 0) salary
                    from
                        x
                    where 
                        not (salary_from is not null and salary_to is not null) and
                        (salary_from is not null or salary_to is not null)
                    
                    union all
                    
                    select
                        *,
                        (coalesce(salary_from, 0) + coalesce(salary_to, 0))/2
                    from
                        x
                    where 
                        (salary_from is not null and salary_to is not null)
                )
                ------------
                
                select
                    e.name,
                    count(k.id) cnt,
                    round(avg(salary),-3) salary
                from
                    k k
                    join employers e on e.id = k.employer_id
                group by 
                    e.name
                order by
                    cnt desc,
                    salary desc
                limit 20
            '''

df = pd.DataFrame(connection.connect().execute(sql_text(query_7_2)))
print('Количествов вакансий DS с указанием заработной платы и средняя заработная плата:')
df

Количествов вакансий DS с указанием заработной платы и средняя заработная плата:


,name,cnt,salary
0,Andersen,3,269000
1,IntelligiChain,3,234000
2,JSA Group,2,300000
3,НСК ЭкоСфера,2,150000
4,РКК Энергия,2,130000
5,Программный регион,2,128000
6,Homework,2,90000
7,DNS Головной офис,2,85000
8,Итон,1,450000
9,AppQuantum,1,350000


In [110]:
# текст запроса
query_7_3 = f'''
                with x as
                (
                    select
                        *
                    from
                        vacancies v
                    where 
                        lower(v.name) like '%data scientist%' or
                        lower(v.name) like '%data science%' or
                        lower(v.name) like '%исследователь данных%' or
                        lower(v.name) like '%machine learning%' or
                        lower(v.name) like '%машинн%обучен%' or
                        (v.name like '%ML%' and v.name not like '%HTML%')
                ),
                -------------------
                
                k as
                (
                    select
                        ar.name "Регион",
                        count(x.id) cnt
                    from
                        x x
                        join areas ar on ar.id = x.area_id
                    group by 
                        ar.name
                    order by
                        cnt desc,
                        ar.name
                    
                )
                
                select 
                    * 
                from 
                    k
                limit 10
            '''

df = pd.DataFrame(connection.connect().execute(sql_text(query_7_3)))
print('Количество вакансий DS по регионам:')
df

Количество вакансий DS по регионам:


,Регион,cnt
0,Москва,217
1,Санкт-Петербург,64
2,Новосибирск,23
3,Нижний Новгород,20
4,Алматы,16
5,Казань,16
6,Минск,9
7,Томск,8
8,Екатеринбург,7
9,Армения,6


In [111]:
# текст запроса
query_7_4 = f'''
                -- Вакансии для DS
                
                with x as
                (
                    select
                        *
                    from
                        vacancies v
                    where 
                        lower(v.name) like '%data scientist%' or
                        lower(v.name) like '%data science%' or
                        lower(v.name) like '%исследователь данных%' or
                        lower(v.name) like '%machine learning%' or
                        lower(v.name) like '%машинн%обучен%' or
                        (v.name like '%ML%' and v.name not like '%HTML%')
                ),
                -------------------
                
                -- Вакансии для начинающих DS
                
                t as
                (
                    select 
                        *
                    from
                        x 
                    where
                        lower(name) like '%junior%' or
                        experience like 'Нет опыта' or
                        employment like 'Стажировка'
                )
                
                -- Группировка по городам от количества
                
                select
                    ar.name "Регион",
                    count(x.id) cnt
                from
                    t x
                    join areas ar on ar.id = x.area_id
                group by 
                    ar.name
                order by
                    cnt desc,
                    ar.name
            '''

df = pd.DataFrame(connection.connect().execute(sql_text(query_7_4)))
print('Количество вакансий начинающих DS по регионам:')
df

Количество вакансий начинающих DS по регионам:


,Регион,cnt
0,Москва,19
1,Санкт-Петербург,9
2,Новосибирск,5
3,Воронеж,2
4,Екатеринбург,2
5,Казань,2
6,Минск,2
7,Пермь,2
8,Ростов-на-Дону,2
9,Томск,2


### Вывод:
    1. Видно, что средняя заработная плата начинающего DS почти в два раза выше, чем в среднем по всем вакансиям.
    2. Среди всех вакансия для DS, почти 10% от компании СБЕР.
    3. Яндекс, несмотря на большое количесвто вакансий на платформе (почти 2000), для DS открыла только 9 вакансий.
    4. Множество вакансий для DS предоставляются банковским сектором.
    5. Однако очень в малом количества вакансий указана заработная плата.
    6. Большинство вакансий для DS сконцентрирована в городах Москва, Санкт-Петербург, Новосибирск.

### Итоги:
    Текущая БД охватывает все регионы России и некторые сторонние регионы ближнего и дальнего зарубежья.
    Среди всего многообразия профессий представленных в БД, вакансии для DS около 1 %,
    а вакансии которые подходят начинающему, Junior, десятая часть от этого процента и составляет 51 вакансия.
    Несммотря на то что Яндекс является техногигантом, он практически не предоставляет вакансий для DS.
    Компаниями которые заинтересованы в специалистах DS являются компаниями из банковского сектора.
    Что касается заработной платы. Во-первых, она указана в менее чем половине всех вакансий в БД.
    Во-вторых, для Junior DS в среднем ЗП почти вдвое выше чем для всех вакансий "Стажировка".
    В-третьих, в среднем удаленная работа оплачивается на 50% больше чем полная занятость.
    Подавляющее большинство вакансий для DS сконцентрирована в городах Москва, Санкт-Петербург.
    Если расматривать DS Junior, то они так же сконцентированы в городах Москва и Санкт-Петербург.